<a href="https://colab.research.google.com/github/HariharanGopinath/Custom-training-of-YOLOV3-model-for-object-detection/blob/main/Train_YoloV3_Multiple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Custom training of YOLOV3 model for object detection**






Here, I have trained the yolo model for 2 classes (wearing mask and not wearing mask). For training this model, I have used the darknet files from the internet and did modification in cfg files for 2 classes object detection




 **Connect google drive**

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Fri Jul 16 09:35:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 chalmers	   'Getting started.pdf'  'My Drive'	'sweden embassy'
'Colab Notebooks'  'Google back up'	  'summer job'	 yolov3


**1) Clone, configure & compile Darknet**
This yolo object detection utilzies the darknet on it's back ground


In [3]:
# Clone
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15257, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 15257 (delta 30), reused 43 (delta 24), pack-reused 15186
Receiving objects: 100% (15257/15257), 13.70 MiB | 25.09 MiB/s, done.
Resolving deltas: 100% (10341/10341), done.


For training our model, we are going to use the GPU,Opencv, and CUDNN. So, we need to give it as those things equal to one in the makefile to use it in the training of our model

In [4]:
# Configure
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


To compile the changes that we have made above

In [ ]:
# Compile
!make

**2) Configure yolov3.cfg file**

In [6]:
# Make a copy of yolov3.cfg
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

This .cfg file consist of all the hyper parameters that are required for training our Yolo model. We can changes those hyper paramters based on the number of classes that we want to train our yolo model

In [7]:
# Change lines in yolov3.cfg file
# We have 2 clasess
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=21@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=21@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=21@' cfg/yolov3_training.cfg

**3) Create .names and .data files**

In [8]:
!echo -e 'without Mask\nwith mask' > data/obj.names
!echo -e 'classes= 2\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data

**4) Save yolov3_training.cfg and obj.names files in Google drive**

In [9]:
!cp cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_testing.cfg
!cp data/obj.names /mydrive/yolov3/classes.txt 

**5) Create a folder and unzip image dataset**

In [ ]:
!mkdir data/obj
!unzip /mydrive/yolov3/images.zip -d data/obj

**6) Create train.txt file**

In [11]:
import glob
images_list = glob.glob("data/obj/images/*.png")
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))

**7) Download pre-trained weights for the convolutional layers file**

In [12]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-07-16 09:40:10--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  42.2MB/s    in 4.0s    

2021-07-16 09:40:15 (38.4 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**8) Start training**

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show
# Uncomment below and comment above to re-start your training from last saved weights
#!./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_training_last.weights -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.859716), count: 16, class_loss = 0.279509, iou_loss = 0.301598, total_loss = 0.581107 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000067, iou_loss = 0.000000, total_loss = 0.000067 
 total_bbox = 390302, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.904195), count: 3, class_loss = 0.064224, iou_loss = 0.017737, total_loss = 0.081961 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.871780), count: 7, class_loss = 0.292383, iou_loss = 0.149769, total_loss = 0.442151 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.675366), count: 6, class_loss = 0.948054, iou_loss = 0.390648, total_loss = 1.338702 
 total_bbox = 390317, rewritten_bbox = 0.000000 % 
v3 (